In [1]:
import os, shutil
import random
import argparse
import torch
import math
import numpy as np
import pandas as pd
import wandb
from lightly.loss.ntx_ent_loss import NTXentLoss
import time
from sklearn.svm import SVC
import seaborn as sns
import glob
import umap.umap_ as umap
import plotly.express as px
import plotly.io as pio
import csv
import itertools

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision.transforms as transforms
from torchvision.models import resnet50, resnet18
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from models.dgcnn import DGCNN, ResNet, DGCNN_partseg
from util import IOStream, AverageMeter
from vis_utils import *
from losses import SupConLoss
import datasets.data_utils as d_utils

import biotite.structure.io as strucio
import biotite.structure as struc
from scipy.spatial.transform import Rotation
from scipy.stats import gaussian_kde
from Bio import PDB
from Bio.PDB import MMCIFParser
from Bio.PDB.vectors import Vector
import open3d as o3d
import warnings
from tqdm import tqdm
from PIL import Image
import datasets.filters as filters

import mrcfile
from scipy.interpolate import RegularGridInterpolator
from scipy.spatial.transform import Rotation
from typing import Tuple

from torch.utils.data import ConcatDataset, DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold

warnings.simplefilter('ignore', PDB.PDBExceptions.PDBConstructionWarning)

parser = argparse.ArgumentParser(description='Model Analysis')
parser.add_argument('--num_points', type=int, default=1024,
                    help='num of points to use')
parser.add_argument('--emb_dims', type=int, default=1024, metavar='N',
                    help='Dimension of embeddings')
parser.add_argument('--k', type=int, default=15, metavar='N',
                        help='Num of nearest neighbors to use')
parser.add_argument('--dropout', type=float, default=1.0,
                        help='dropout rate')
parser.add_argument('--load_perc', type=float, default=1.0, help='The percentage of data to be loaded onto the RAM')
args = parser.parse_args("")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
rng = random.Random(0)

Trained model folder names:

* crosspoint_newFSData_simclr_noNorm_noise2d_0
* crosspoint_newFSData_simclr_noNorm_noise2d_0_0.01_0.03_0.05

In [3]:
data_dir = "/global/scratch/users/kithminiherath/in_situ_2d2d_realdata_binned_2t"
exp_name = "cl_2d2d_realbinneddata_resnet50_HoriflipVerflip"
filter_type = "lowpass"
filter_cutoff = 0.2
label_map = {0:'ribosome',1:'background'}

In [4]:
def make_dir(fp):
    if os.path.isdir(fp):
        shutil.rmtree(fp)

    os.mkdir(fp)

# Load models

In [5]:
device = "cuda:0"
img_model_dict = torch.load(f'/global/scratch/users/kithminiherath/results/{exp_name}/models/img_model_best.pth', map_location="cuda:0")

img_model = ResNet(resnet50(), feat_dim = 2048).to(device)

img_model.load_state_dict(img_model_dict)

<All keys matched successfully>

# Get train/ val features

In [6]:
def load_img_data(DATA_DIR):
    all_filepath = []
    
    for cls in sorted(glob.glob(os.path.join(DATA_DIR, '*'))):
        imgs = sorted(glob.glob(os.path.join(cls, '*')))
        if len(imgs)%2 == 0:
            all_filepath += imgs
        else:
            all_filepath += imgs[:-1]
    # print(all_filepath)    
    return all_filepath

def load_img_file(fp):
    with mrcfile.open(fp) as mrc:
        data = mrc.data.copy()
        
    return data

def prenorm(fp):
    if "_projection_" not in fp:
        img = load_img_file(fp)
        return (img - np.mean(img)) / np.std(img)
    else:
        return load_img_file(fp)

class cl_data_loader(Dataset):
    def __init__(self, fp = "", img_transform = None, type_="train", filter_type="lowpass", filter_cutoff=0.1, fourier_transformed=False):
        self.data = load_img_data(fp) # ribosome, background
        self.transform = img_transform
        self.type = type_
        self.filter_type = filter_type
        self.filter_cutoff = filter_cutoff
        self.fourier_transformed = fourier_transformed
    
    def __getitem__(self, item):                
        label_map = {'ribosome':0, 'background':1}
        
        img1_fp = self.data[2*item]
        img2_fp = self.data[2*item + 1]
        
        img1_class = img1_fp.split("/")[-2]
        img2_class = img2_fp.split("/")[-2]
                
        assert img1_class == img2_class, "Classes different in positive pair !!!"
        
        img1 = prenorm(img1_fp)
        img2 = prenorm(img2_fp)
        
        img1_filtered = filters.filter_image(img1, self.filter_cutoff, filter_type = self.filter_type)
        img2_filtered = filters.filter_image(img2, self.filter_cutoff, filter_type = self.filter_type)
        
        if self.fourier_transformed:
            img1_filtered = np.fft.fftshift(np.fft.fft2(img1_filtered))
            img2_filtered = np.fft.fftshift(np.fft.fft2(img2_filtered))

        img_t1 = self.transform(img1_filtered.astype('float32'))
        img_t2 = self.transform(img2_filtered.astype('float32'))
        
        if "projection" in img1_fp.split("/")[-1][:-4]:
            ftype = "simulated"
        else:
            ftype = "real"
    
        return img_t1, img_t2, label_map[img1_class], ftype
    
    def __len__(self):
        return len(self.data)//2

In [7]:
def get_features(img_model, loader, device="cuda", type_="train"):
    img_feats_arr = []
    labels_arr = []
    fnames_arr = []
    print(device)
    
    for i, (imgs1, imgs2, labels, fnames) in enumerate(loader):
        if i>60: break
        imgs = imgs1.to(device)
        labels = labels.numpy().tolist()
        
        with torch.no_grad():
            img_feats = img_model.resnet(imgs)
        
        img_feats = F.normalize(img_feats, dim=1).detach().cpu().numpy()
        img_feats_arr.extend(img_feats)
        labels_arr += labels
        fnames_arr += fnames
                                
    return np.array(img_feats_arr), np.array(labels_arr), np.array(fnames_arr)

# Classification

Neg class label map:
* proteasome = 1
* apoferritin = 2
* betagal = 3

In [8]:
import scipy.stats as stats

def get_bar_plots_binaryclass(ribosome_scores, neg_scores):
    barWidth = 0.38
    ribo_means = {key: np.mean(values) for key, values in ribosome_scores.items()}
    ribo_std_errors = {key: np.std(values, ddof=1) / np.sqrt(len(values)) 
                 for key, values in ribosome_scores.items()}
    neg_means = {key: np.mean(values) for key, values in neg_scores.items()}
    neg_std_errors = {key: np.std(values, ddof=1) / np.sqrt(len(values)) 
                 for key, values in neg_scores.items()}
    
    labels = [key.split('_')[-1] for key in list(ribosome_scores.keys())]
    ribo_mean_values = list(ribo_means.values())
    ribo_std_error_values = list(ribo_std_errors.values())
    neg_mean_values = list(neg_means.values())
    neg_std_error_values = list(neg_std_errors.values())
    
    fig, ax = plt.subplots(figsize = (12, 8))  

    # Set position of bar on X axis 
    br1 = np.arange(len(labels)) 
    br2 = [x + barWidth for x in br1] 
    
    # Set the silver/gray background color
    ax.set_facecolor('#EAEAF0')  # Light gray color
    fig.patch.set_facecolor('white')
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    # Define error bar style
    error_params = dict(elinewidth=3, capsize=10, capthick=1.5, ecolor='black')

    # Make the plot
    plt.bar(br1, ribo_mean_values, yerr=ribo_std_error_values, color ='#384860', width = barWidth, 
            edgecolor ='none', label ='Ribosome', zorder=2, error_kw=error_params) # , yerr=[train_errs0, train_errs1], capsize=5
    plt.bar(br2, neg_mean_values, yerr=neg_std_error_values, color ='#97a6c4', width = barWidth, 
            edgecolor ='none', label ='Negative class', zorder=2, error_kw=error_params) # , yerr=[val_errs0, val_errs1], capsize=5

    plt.grid(True, axis='y', color='black', linestyle='--', linewidth=0.5, zorder=0)
    
    # Adding Xticks 
    plt.xlabel('Noise (std)', fontweight ='bold', fontsize = 25) 
    plt.ylabel('Accuracy', fontweight ='bold', fontsize = 25) 
    plt.xticks([r + barWidth/2 for r in range(len(labels))], labels, fontsize=20)
    plt.yticks(fontsize=20)
    
    # plt.title(title)
    # plt.legend()
    # plt.savefig(f'/hpc/projects/group.czii/kithmini.herath/crosspoint-analysis/first_experiments_multipleclasses_pointmodel_uniqueanglesubsets/{caption}_bar.pdf', dpi=300, bbox_inches='tight')
    plt.show() 
    
def plot_bar_graph_crossval(scores_dict):
    # Calculate means and standard errors
    barWidth = 0.3
    means = {key: np.mean(values) for key, values in scores_dict.items()}
    std_errors = {key: np.std(values, ddof=1) / np.sqrt(len(values)) 
                 for key, values in fold_scores.items()}

    # Prepare data for plotting
    labels = [key.split('_')[-1] for key in list(fold_scores.keys())]
    mean_values = list(means.values())
    std_error_values = list(std_errors.values())
    
    # Define error bar style
    error_params = dict(elinewidth=3, capsize=10, capthick=1.5, ecolor='black')

    # Create the bar plot
    x = np.arange(len(labels))
    fig, ax = plt.subplots(figsize = (12, 8))
    # Set the silver/gray background color
    ax.set_facecolor('#EAEAF0')  # Light gray color
    fig.patch.set_facecolor('white')
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    ax.bar(x, mean_values, yerr=std_error_values, capsize=5, color='#f1a226', zorder=2, error_kw=error_params)
    
    plt.grid(True, axis='y', color='black', linestyle='--', linewidth=0.5, zorder=0)

    # Customize the plot
    plt.xlabel('Noise (std)', fontweight ='bold', fontsize = 25) 
    plt.ylabel('Accuracy', fontweight ='bold', fontsize = 25) 
    plt.xticks(x, labels, fontsize=20)
    plt.yticks(fontsize=20)

    plt.show()

In [9]:
# Classifier train codes

def get_class_accs(cls_dict, preds, labels):
    # label_map = {0:'ribosome', 1:'atpase', 2:'proteasome'}

    for idx, key in enumerate(cls_dict.keys()):
        # print(idx, key)
        cls_idxs = np.where(np.array(labels) == idx)[0]
        cls_preds = np.array(preds)[cls_idxs]
        cls_y = np.array(labels)[cls_idxs]
        cls_dict[key] = (cls_preds == cls_y).sum()/len(cls_idxs)
    return cls_dict

class classifier(nn.Module):
    """
    A simple classifier.

    Attributes:
    num_classes: The number of classes to classify into.
    kernel_initializer: An initializer to use for the weights.
    """
    
    def __init__(self, num_classes, kernel_initializer=None):
        super(classifier, self).__init__()
        self.num_classes = num_classes
        
        # Define the dense layer
        self.dense_layer = nn.Linear(in_features=2048, out_features=num_classes)
        
        # Initialize weights if initializer is provided
        if kernel_initializer:
            kernel_initializer(self.dense_layer.weight)
    
    def forward(self, inputs):
        # Ensure the input has rank 2
        if len(inputs.shape) != 2:
            raise ValueError(f'Input shape {inputs.shape} is expected to have rank 2, but does not.')
        
        return self.dense_layer(inputs)

def loop(model, feats, labels, opt, criterion, epoch, type_ = "train", device="cuda:0"):
    losses = AverageMeter()
    total, correct = 0, 0

    if type_ == "train":
        model.train()
        # print(f'Start training epoch')
    elif type_ == "val":
        model.eval()
        # print(f'Start validation')
    
    bs = 32
    nb = feats.shape[0]//bs
    # print(feats.shape, labels.shape)
    for i in range(nb):
        x = torch.from_numpy(feats[i*bs: (i+1)*bs]).to(device)
        y = torch.from_numpy(labels[i*bs: (i+1)*bs]).to(device)

        if type_ == "train":
            opt.zero_grad()
            preds = model(x)
        elif type_ == "val":
            with torch.no_grad():
                preds = model(x)

        loss = criterion(preds, y)   # logits, targets
        
        _, y_pred = torch.max(preds, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()
        
        if type_ == "train":
            loss.backward()
            opt.step()

        losses.update(loss.item(), bs)
            
    return losses, correct/total

def train(train_data, val_data, lr, epochs, device="cuda:0"):
    # print("train device:",device)
    train_feats, train_labels = train_data
    val_feats, val_labels = val_data
    torch.manual_seed(0)
    
    if not os.path.exists('/global/scratch/users/kithminiherath/results/cross-validation-realdata'):
        os.makedirs('/global/scratch/users/kithminiherath/results/cross-validation-realdata')
    if not os.path.exists('/global/scratch/users/kithminiherath/results/cross-validation-realdata/models'):
        os.makedirs('/global/scratch/users/kithminiherath/results/cross-validation-realdata/models')
    
    model = classifier(num_classes=2, kernel_initializer=nn.init.xavier_uniform_).to(device)
    
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
    
    criterion = nn.CrossEntropyLoss().to(device)
    
    train_loss, val_loss = [], []
    train_acc, val_acc = [], []
    for epoch in range(epochs):
        train_losses, train_accs = loop(model, train_feats, train_labels, opt, criterion, epoch, type_ = "train", device=device)
        val_losses, val_accs = loop(model, val_feats, val_labels, opt, criterion, epoch, type_ = "val", device=device)
        
        train_loss.append(train_losses.avg)
        val_loss.append(val_losses.avg)
        
        train_acc.append(train_accs)
        val_acc.append(val_accs)
        
        if epoch % 10 == 0:
            # print('==> Saving...')
            save_file = os.path.join(f'/global/scratch/users/kithminiherath/results/cross-validation-realdata/models/','ckpt_epoch_{epoch}.pth'.format(epoch=epoch))
            torch.save(model.state_dict(), save_file)


    # print('==> Saving Last Model...')
    save_file = os.path.join(f'/global/scratch/users/kithminiherath/results/cross-validation-realdata/models/',
                             'ckpt_epoch_last.pth'.format(epoch=epoch))
    torch.save(model.state_dict(), save_file)

In [ ]:
noise_2d = [0.0]

data_dir = "/global/scratch/users/kithminiherath/in_situ_2d2d_realdata_binned_2t"
label_map = {0:'ribosome',1:'background'}

img_model = img_model.to(device)
img_model = img_model.eval()

# Store scores
fold_scores = {
    "noise_0.0" : []
}

fold_scores_ribosome = {
    "noise_0.0" : []
}

fold_scores_neg_class = {
    "noise_0.0" : []
}

for noise in noise_2d:
    print(f"Noise level = {noise} *************************")
    transform = transforms.Compose([transforms.ToTensor()]) #, transforms.CenterCrop((300, 300))

    type_ = "train"
    train_set = cl_data_loader(f"{data_dir}/{type_}", img_transform = transform, type_=type_, filter_type=filter_type, filter_cutoff=filter_cutoff)

    type_ = "test"
    val_set = cl_data_loader(f"{data_dir}/{type_}", img_transform = transform, type_=type_, filter_type=filter_type, filter_cutoff=filter_cutoff)
    
    combined_dataset = ConcatDataset([train_set, val_set])
    
    # Initialize K-Fold
    kfold = KFold(n_splits=10, shuffle=True, random_state=10)

    ########## Feature Extraction
    for fold, (train_ids, val_ids) in enumerate(kfold.split(combined_dataset)):
        # print(f'FOLD {fold + 1}')
        
        # Create samplers for train and validation splits
        train_sampler = SubsetRandomSampler(train_ids)
        val_sampler = SubsetRandomSampler(val_ids)
        
        # Create data loaders with your specific parameters
        train_loader = DataLoader(
            combined_dataset,
            batch_size=32,
            sampler=train_sampler,
            num_workers=12,
            pin_memory=True,
            drop_last=True
        )
        
        val_loader = DataLoader(
            combined_dataset,
            batch_size=32,
            sampler=val_sampler,
            num_workers=12,
            pin_memory=True,
            drop_last=True
        )
        
        # normalized features
        img_feats_train, labels_train, type_train = get_features(img_model, train_loader, device=device, type_="train")
        # print(f"PC features (train) shape: {point_feats_train.shape} | Image features (train) shape: {img_feats_train.shape}")
        # print(img_feats_train.shape, labels_train.shape, fnames_train.shape)
        # print(labels_train, len(labels_train))

        # normalized features
        img_feats_val, labels_val, type_val = get_features(img_model, val_loader, device=device, type_="val")
        # print(f"PC features (test) shape: {point_feats_val.shape} | Image features (test) shape: {img_feats_val.shape}")
        # print(point_feats_val.shape, img_feats_val.shape, labels_val.shape, fnames_val.shape)
        # print(labels_val, len(labels_val))
    
        ######## Train classifier
        lr = 0.001
        epochs = 40
        # device = torch.device("cuda:0")

        train((img_feats_train, labels_train), (img_feats_val, labels_val), lr, epochs, device=device)

        classifier_dict = torch.load(f'/global/scratch/users/kithminiherath/results/cross-validation-realdata/models/ckpt_epoch_last.pth', map_location="cuda:0")
        classifier_model = classifier(num_classes=2, kernel_initializer=nn.init.xavier_uniform_).to(device)

        classifier_model.load_state_dict(classifier_dict)

        classifier_model.eval()

        with torch.no_grad():
            preds_img_val = classifier_model(torch.from_numpy(img_feats_val).to(device)).to(device)

        _, nn_img_val_preds = torch.max(preds_img_val, 1)

        total_val = len(labels_val)

        nn_img_val_acc = ((nn_img_val_preds.detach().cpu().numpy() == labels_val).sum().item())/total_val
        
        fold_scores[f"noise_{noise}"].append(nn_img_val_acc)
        
        cls_nn_img_val_acc = get_class_accs({'ribosome':0, 'background':0}, nn_img_val_preds.detach().cpu().numpy(), labels_val)
        
        fold_scores_ribosome[f"noise_{noise}"].append(cls_nn_img_val_acc['ribosome'])
        fold_scores_neg_class[f"noise_{noise}"].append(cls_nn_img_val_acc['background'])

plot_bar_graph_crossval(fold_scores)
get_bar_plots_binaryclass(fold_scores_ribosome, fold_scores_neg_class)

Noise level = 0.0 *************************
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


In [ ]:
np.mean(fold_scores['noise_0.0']), np.mean(fold_scores_ribosome['noise_0.0']), np.mean(fold_scores_neg_class['noise_0.0'])


# SVC

In [ ]:
noise_2d = [0.0]

data_dir = "/global/scratch/users/kithminiherath/in_situ_2d2d_realdata_binned_2t"
label_map = {0:'ribosome',1:'background'}

img_model = img_model.to(device)
img_model = img_model.eval()

# Store scores
fold_scores = {
    "noise_0.0" : []
}

fold_scores_ribosome = {
    "noise_0.0" : []
}

fold_scores_neg_class = {
    "noise_0.0" : []
}

for noise in noise_2d:
    print(f"Noise level = {noise} *************************")
    transform = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop((300, 300))])

    type_ = "train"
    train_set = cl_data_loader(f"{data_dir}/{type_}", img_transform = transform, type_=type_, filter_type=filter_type, filter_cutoff=filter_cutoff)

    type_ = "test"
    val_set = cl_data_loader(f"{data_dir}/{type_}", img_transform = transform, type_=type_, filter_type=filter_type, filter_cutoff=filter_cutoff)
    
    combined_dataset = ConcatDataset([train_set, val_set])
    
    # Initialize K-Fold
    kfold = KFold(n_splits=10, shuffle=True, random_state=10)

    ########## Feature Extraction
    for fold, (train_ids, val_ids) in enumerate(kfold.split(combined_dataset)):
        # print(f'FOLD {fold + 1}')
        
        # Create samplers for train and validation splits
        train_sampler = SubsetRandomSampler(train_ids)
        val_sampler = SubsetRandomSampler(val_ids)
        
        # Create data loaders with your specific parameters
        train_loader = DataLoader(
            combined_dataset,
            batch_size=32,
            sampler=train_sampler,
            num_workers=12,
            pin_memory=True,
            drop_last=True
        )
        
        val_loader = DataLoader(
            combined_dataset,
            batch_size=32,
            sampler=val_sampler,
            num_workers=12,
            pin_memory=True,
            drop_last=True
        )
        
        # normalized features
        img_feats_train, labels_train, type_train = get_features(img_model, train_loader, device=device, type_="train")
        # print(f"PC features (train) shape: {point_feats_train.shape} | Image features (train) shape: {img_feats_train.shape}")
        # print(img_feats_train.shape, labels_train.shape, fnames_train.shape)
        # print(labels_train, len(labels_train))

        # normalized features
        img_feats_val, labels_val, type_val = get_features(img_model, val_loader, device=device, type_="val")
        # print(f"PC features (test) shape: {point_feats_val.shape} | Image features (test) shape: {img_feats_val.shape}")
        # print(point_feats_val.shape, img_feats_val.shape, labels_val.shape, fnames_val.shape)
        # print(labels_val, len(labels_val))
    
        ######## Fit classifier on train set
        model_tl = SVC(C = 0.1, kernel ='linear')

        feats_train = np.array(img_feats_train)
        labels_train = np.array(labels_train)

        model_tl.fit(feats_train, labels_train) 
        
        feats_val = np.array(img_feats_val)
        labels_val = np.array(labels_val)
        
        val_accuracy = model_tl.score(feats_val, labels_val)
        fold_scores[f"noise_{noise}"].append(val_accuracy)
        
        preds_img_val = model_tl.predict(feats_val)
                
        total_val = len(labels_val)
        
        cls_img_val_acc = get_class_accs({'ribosome':0, 'background':0}, preds_img_val, labels_val)
        
        fold_scores_ribosome[f"noise_{noise}"].append(cls_img_val_acc['ribosome'])
        fold_scores_neg_class[f"noise_{noise}"].append(cls_img_val_acc['background'])

np.mean(fold_scores['noise_0.0']), np.mean(fold_scores_ribosome['noise_0.0']), np.mean(fold_scores_neg_class['noise_0.0'])